In [ ]:
!pip install tensorflow

In [3]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm #to check loop progress
tqdm().pandas()

/var/folders/rt/0kf7v29569z7rctzmpst_dsh0000gn/T/ipykernel_20296/429505501.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [61]:
# Load the document file into memory
def load_fp(file):
  # Open file to read
  with open(file, 'r') as f:
    text = f.read()
    return text # string

# Get all images with their captions
def img_capt(filename):
    # call the load_fp function directly
    if filename.endswith('.txt'):
      file = load_fp(filename)
      # Split the string into lines
      lines = file.split("\n")

      # Initialize dictionaries to store images and captions
      image_to_captions = {}

      # Process each line
      for line in lines:
          # Skip empty or improperly formatted lines
          if not line.strip() or "\t" not in line:
              continue

          # Split the line into image and caption
          image_caption, caption = line.split("\t")
          image, _ = image_caption.split("#")  # Extract the image name (ignore the #index)

          # Add the caption to the corresponding image
          if image not in image_to_captions:
              image_to_captions[image] = []
          image_to_captions[image].append(caption)

      # Separate the images and captions into two lists
      images = list(image_to_captions.keys())
      captions = list(image_to_captions.values())

      # Output the results
    #   print("Images:", images)
    #   print("Captions:", captions)
      return image_to_captions # dictionary

In [96]:
# Convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
def txt_clean(captions):
  table = str.maketrans('','', string.punctuation)
  for img,caps in captions.items():
    for i,img_caption in enumerate(caps):
        img_caption.replace("-"," ")
        descp = img_caption.split()
        #uppercase to lowercase
        descp = [wrd.lower() for wrd in descp]
        #remove punctuation from each token
        descp = [wrd.translate(table) for wrd in descp]
        #remove hanging 's and a
        descp = [wrd for wrd in descp if(len(wrd)>1)]
        #remove words containing numbers with them
        descp = [wrd for wrd in descp if(wrd.isalpha())]
        #converting back to string
        img_caption = ' '.join(descp)
        captions[img][i]= img_caption
  return captions

# To build a set of all unique words
def txt_vocab(descriptions):
  vocab = set()
  for key in descriptions.keys():
    [vocab.update(d.split()) for d in descriptions[key]]
  return vocab

# To save all descriptions in one file
def save_descriptions(descriptions, filename):
    """
    Save all descriptions to a file in the format:
    image_id \t caption
    - descriptions: Dictionary of image IDs and their captions.
    - filename: Path to the file where descriptions will be saved.
    """
    lines = []
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            # Use a tab character (\t) to separate the image ID and caption
            lines.append(key + "\t" + desc)

    # Join all lines with a newline character (\n)
    data = "\n".join(lines)

    # Write the data to the file
    with open(filename, "w") as file:
        file.write(data)

In [ ]:
# Set these path according to project folder in you system, like i create a folder with my name shikha inside D-drive
dataset_text = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/Flickr8k_text"
dataset_images = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/Flicker8k_Dataset"

# to prepare our text data: just one of them for now
filename = dataset_text + "/" + "Flickr8k.token.txt"

# loading the file that contains all data
# map them into descriptions dictionary
description = img_capt('/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/Flickr8k_text/Flickr8k.token.txt')
print("Length of descriptions =" ,len(description))

# cleaning the descriptions
clean_descriptions = txt_clean(description)

# to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

Length of descriptions = 8092
Length of vocabulary =  8763


In [98]:
import tensorflow
import os
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pickle

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    """
    Preprocess a single image for feature extraction using ResNet50.
    - Resizes the image to the target size.
    - Applies ResNet50-specific preprocessing.
    """
    image = load_img(image_path, target_size=target_size)  # Load and resize image
    image = img_to_array(image)  # Convert to numpy array
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = preprocess_input(image)  # Preprocess for ResNet50
    return image

def extract_features(directory, model=None):
    """
    Extract features for all images in a directory using a pre-trained ResNet50 model.
    - directory: Path to the directory containing images.
    - model: Pre-trained model for feature extraction (default: ResNet50).
    """
    if model is None:
        # Load the ResNet50 model without the top layer
        model = ResNet50(include_top=False, pooling='avg')

    features = {}
    for img_name in tqdm(os.listdir(directory), desc="Extracting Features"):
        img_path = os.path.join(directory, img_name)
        try:
            # Preprocess the image
            image = preprocess_image(img_path)
            # Extract features using the model
            feature = model.predict(image, verbose=0)
            # Store the feature vector
            features[img_name] = feature
        except Exception as e:
            print(f"Error processing {img_name}: {e}")
    return features

# Save features to a pickle file
def save_features(features, filename="features.p"):
    with open(filename, "wb") as file:
        pickle.dump(features, file)

# Load features from a pickle file
def easy_load_features(filename="features.p"):
    with open(filename, "rb") as file:
        return pickle.load(file)

# Example usage
dataset_images = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/Flicker8k_Dataset"
features = extract_features(dataset_images)
loaded_features = easy_load_features("features.p")

In [106]:
clean_descriptions

{'1000268201_693b08cb0e.jpg': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70.jpg': ['black dog and spotted dog are fighting',
  'black dog and tricolored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8.jpg': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young 

In [ ]:
# saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")
# saving all features in one file
save_features(features, "features.p")

#### START HERE

In [112]:
def load_features(photos, filename="features.p"):
    """
    Load features from a pickle file and filter them for the given photos.
    - photos: List of photo IDs to filter features for.
    - filename: Path to the pickle file containing all features.
    Returns: A dictionary of features for the given photos.
    """
    # Load all features from the pickle file
    with open(filename, "rb") as file:
        all_features = pickle.load(file)

    # Filter features for the given photos
    features = {photo: all_features[photo] for photo in photos if photo in all_features}
    return features

def load_photos(filename):
    """
    Load a list of photo IDs from a file.
    - filename: Path to the file containing photo IDs.
    Returns: A list of photo IDs.
    """
    file = load_fp(filename)  # Load the file content
    photos = file.split("\n")[:-1]  # Split into lines and remove the last empty line
    return photos

def load_clean_descriptions(filename, photos):
    """
    Load and clean descriptions for the given photos.
    - filename: Path to the file containing descriptions.
    - photos: List of photo IDs to filter descriptions for.
    Returns: A dictionary of descriptions where keys are photo IDs and values are lists of captions.
    """
    # Load the file content
    file = load_fp(filename)
    descriptions = {}

    # Process each line in the file
    for line in file.split("\n"):
        words = line.split()
        if len(words) < 1:  # Skip empty lines
            continue

        # Extract image ID and caption
        image, image_caption = words[0], words[1:]

        # Only include descriptions for photos in the given list
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            # Join the caption words and add to the descriptions dictionary
            desc = " ".join(image_caption)
            descriptions[image].append(desc)

    return descriptions

In [139]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

# Convert dictionary of descriptions to a list of all descriptions
def dict_to_list(descriptions):
    """
    Convert a dictionary of image descriptions into a list of all descriptions.
    - descriptions: Dictionary where keys are image IDs and values are lists of captions.
    """
    all_desc = []
    for key in descriptions.keys():
        all_desc.extend(descriptions[key])  # Add all captions for each image
    return all_desc

# Create a tokenizer to vectorize the text corpus
def create_tokenizer(descriptions):
    """
    Create a tokenizer for the given descriptions.
    - descriptions: Dictionary of image descriptions.
    Returns: A fitted Tokenizer object.
    """
    desc_list = dict_to_list(descriptions)  # Convert dictionary to list of descriptions
    tokenizer = Tokenizer(oov_token="<unk>")  # Initialize the tokenizer
    tokenizer.fit_on_texts(desc_list)  # Fit the tokenizer on the text data
    return tokenizer

# Save the tokenizer to a pickle file
def save_tokenizer(tokenizer, filename="tokenizer.p"):
    """
    Save the tokenizer object to a pickle file.
    - tokenizer: Fitted Tokenizer object.
    - filename: Name of the file to save the tokenizer.
    """
    with open(filename, "wb") as file:
        pickle.dump(tokenizer, file)

# Calculate the maximum length of descriptions
def max_length(descriptions):
    """
    Calculate the maximum length of descriptions.
    - descriptions: Dictionary of image descriptions.
    Returns: The maximum length of any description.
    """
    desc_list = dict_to_list(descriptions)  # Convert dictionary to list of descriptions
    return max(len(d.split()) for d in desc_list)  # Find the longest description

In [136]:
import re 

def preprocess_caption(caption):
    # Convert to lowercase
    caption = caption.lower()
    # Remove punctuation
    caption = re.sub(r'[^\w\s]', '', caption)
    # Remove extra whitespace
    caption = caption.strip()
    return caption

In [137]:
# Extract all captions from the dictionary
descriptions_list = []
for captions in train_descriptions.values():  # Replace `descriptions_dict` with your dictionary
    descriptions_list.extend(captions)

# Preprocess the captions
descriptions_list = [preprocess_caption(caption) for caption in descriptions_list]

In [138]:
descriptions_list

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin',
 'black dog and spotted dog are fighting',
 'black dog and tricolored dog playing with each other on the road',
 'black dog and white dog with brown spots are staring at each other in the street',
 'two dogs of different breeds looking at each other on the road',
 'two dogs on pavement moving toward each other',
 'little girl covered in paint sits in front of painted rainbow with her hands in bowl',
 'little girl is sitting in front of large painted rainbow',
 'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
 'there is girl with pigtails sitting in front of rainbow painting',
 'young girl with pigtails painting outside in the grass',
 'man lays on bench while his dog sits by him',
 'man

In [134]:
train_descriptions

{'1000268201_693b08cb0e.jpg': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70.jpg': ['black dog and spotted dog are fighting',
  'black dog and tricolored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8.jpg': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young 

In [140]:
# Assuming `train_descriptions` is already loaded
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

tokenizer = create_tokenizer(train_descriptions)  # Create the tokenizer
save_tokenizer(tokenizer, "tokenizer.p")  # Save the tokenizer to a file

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")  # Example: 7577 words

# Maximum description length
max_len = max_length(train_descriptions)
print(f"Maximum description length: {max_len}")  # Example: 32

Vocabulary size: 7578
Maximum description length: 32


In [122]:
train_images_file = "/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/Flickr8k_text/Flickr_8k.trainImages.txt"
train_imgs = load_photos(filename=train_images_file)
train_features = load_features(train_imgs)
train_descriptions = load_clean_descriptions("/Users/ruhwang/Desktop/AI/spring2025_courses/aipi540-dl/ig_post_generator/descriptions.txt", train_imgs)

In [81]:
train_features

{'2513260012_03d33305cf.jpg': array([[0.3470795 , 0.51159143, 0.08728869, ..., 1.1897292 , 0.04043844,
         0.15271074]], dtype=float32),
 '2903617548_d3e38d7f88.jpg': array([[0.       , 0.003078 , 0.0936   , ..., 0.4310345, 0.3798528,
         0.9165751]], dtype=float32),
 '3338291921_fe7ae0c8f8.jpg': array([[0.7674461 , 0.53422654, 0.38325134, ..., 0.02098666, 0.18429387,
         0.04270008]], dtype=float32),
 '488416045_1c6d903fe0.jpg': array([[6.2481058e-01, 3.2965431e-01, 1.7824951e-04, ..., 1.2753807e+00,
         4.6392533e-01, 4.9036017e-01]], dtype=float32),
 '2644326817_8f45080b87.jpg': array([[0.11555288, 0.6596538 , 0.05323274, ..., 0.14397332, 1.0179372 ,
         0.00556589]], dtype=float32),
 '218342358_1755a9cce1.jpg': array([[1.7061838 , 0.40514764, 0.35206538, ..., 0.2940881 , 0.04833034,
         0.04654434]], dtype=float32),
 '2501968935_02f2cd8079.jpg': array([[0.10608266, 1.4420816 , 0.04446993, ..., 0.20587137, 0.69787616,
         1.5904329 ]], dtype=float3

In [126]:
type(train_features), type(train_descriptions), type(train_imgs), type(tokenizer)

(dict, dict, list, keras.src.legacy.preprocessing.text.Tokenizer)

In [124]:
assert train_descriptions != clean_descriptions

In [127]:
# 2. Example: Convert some new (or existing) text descriptions
texts_to_convert = ['cat on the mat', 'dog in the park']
sequences = tokenizer.texts_to_sequences(texts_to_convert)
sequences


[[650, 3, 2, 1114], [6, 1, 2, 112]]

In [128]:
tokenizer.texts_to_sequences(dict_to_list(train_descriptions))

[[40, 1, 85, 167, 4, 115, 53, 391, 9, 392, 1, 25, 4472, 638],
 [16, 311, 62, 193, 116],
 [37, 16, 115, 62, 193, 2055],
 [37, 16, 115, 2, 392, 17, 58, 2055],
 [37, 16, 1, 85, 167, 311, 62, 193, 2913],
 [12, 6, 5, 851, 6, 14, 339],
 [12, 6, 5, 1550, 6, 31, 7, 135, 81, 3, 2, 144],
 [12, 6, 5, 11, 6, 7, 22, 985, 14, 620, 19, 135, 81, 1, 2, 70],
 [10, 28, 9, 833, 2261, 82, 19, 135, 81, 3, 2, 144],
 [10, 28, 3, 663, 852, 316, 135, 81],
 [37, 16, 179, 1, 609, 102, 1, 46, 9, 558, 1107, 7, 58, 215, 1, 1007],
 [37, 16, 4, 45, 1, 46, 9, 55, 558, 1107],
 [49, 16, 1, 2, 38, 108, 7, 3470, 1, 46, 9, 11, 2529, 7, 1107, 3, 152],
 [180, 4, 16, 7, 1288, 45, 1, 46, 9, 1107, 762],
 [23, 16, 7, 1288, 762, 78, 1, 2, 38],
 [8, 621, 3, 150, 32, 26, 6, 102, 54, 146],
 [8, 621, 3, 2, 150, 17, 677, 11, 6, 4, 1144, 1404],
 [8, 610, 3, 150, 78, 7, 11, 5, 12, 6, 45, 87, 17, 146],
 [420, 8, 834, 3, 112, 150, 7, 26, 6],
 [8, 257, 3, 150, 48, 323, 9, 6, 45, 3, 161],
 [8, 1, 25, 83, 93, 1888, 19, 170],
 [8, 373, 25, 83,

In [131]:
dict_to_list(train_descriptions)

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin',
 'black dog and spotted dog are fighting',
 'black dog and tricolored dog playing with each other on the road',
 'black dog and white dog with brown spots are staring at each other in the street',
 'two dogs of different breeds looking at each other on the road',
 'two dogs on pavement moving toward each other',
 'little girl covered in paint sits in front of painted rainbow with her hands in bowl',
 'little girl is sitting in front of large painted rainbow',
 'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
 'there is girl with pigtails sitting in front of rainbow painting',
 'young girl with pigtails painting outside in the grass',
 'man lays on bench while his dog sits by him',
 'man

In [141]:
import torch.nn as nn
import torch

In [129]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [146]:
def train_word2vec(data):
    model = Word2Vec([sentence.split() for sentence in data], vector_size=100, window=5, min_count=1, workers=-1)
    return model

In [ ]:
from gensim.models import Word2Vec

def train_word2vec(data, vector_size=100, window=5, min_count=1, workers=4, epochs=5):
    """
    Train a Word2Vec model on the given data.
    - data: List of sentences (each sentence is a list of words).
    - vector_size: Dimensionality of the word vectors.
    - window: Maximum distance between the current and predicted word.
    - min_count: Ignores words with total frequency lower than this.
    - workers: Number of worker threads to train the model.
    - epochs: Number of training epochs.
    Returns: Trained Word2Vec model.
    """
    # Tokenize the data into a list of lists of words
    tokenized_data = [sentence.split() for sentence in data]

    # Initialize the Word2Vec model
    model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=workers)

    # Build the vocabulary
    model.build_vocab(tokenized_data)

    # Train the model
    model.train(tokenized_data, total_examples=len(tokenized_data), epochs=epochs)

    return model

# Convert train_descriptions to a list of sentences
train_sentences = dict_to_list(train_descriptions)

# Train the Word2Vec model
word2vec_model = train_word2vec(train_sentences, vector_size=len(train_sentences), window=5, min_count=1, workers=4, epochs=10)

# Save the model
word2vec_model.save("word2vec.model")

In [144]:
def create_embedding_matrix_word2vec(tokenizer, word2vec, embedding_dim):
    """
    Create an embedding matrix using Word2Vec embeddings.

    Args:
        tokenizer: The tokenizer used to tokenize the captions (e.g., Keras Tokenizer).
        word2vec: The pretrained Word2Vec model (gensim Word2Vec object).
        embedding_dim: The dimension of the Word2Vec embeddings.

    Returns:
        A PyTorch tensor containing the embedding matrix.
    """
    vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
    embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))  # Random initialization

    # Get the Word2Vec vocabulary
    word2vec_vocab = set(word2vec.wv.index_to_key)

    for word, i in tokenizer.word_index.items():
        if word in word2vec_vocab:  # Check if the word exists in Word2Vec
            embedding_matrix[i] = word2vec.wv[word]  # Use the pretrained vector
        else:
            # Word not in Word2Vec, keep the random initialization
            pass

    print(f"Embedding matrix created with shape: {embedding_matrix.shape}")
    return torch.tensor(embedding_matrix, dtype=torch.float32)

In [142]:
embedding_dim = 25500

In [147]:
embedding_matrix = create_embedding_matrix_word2vec(tokenizer, word2vec_model, embedding_dim)
embedding_matrix

NameError: name 'word2vec_model' is not defined